# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fale old settlement,-9.3852,-171.2468,27.93,74,28,3.45,TK,1730644858
1,1,pacific grove,36.6177,-121.9166,12.83,92,0,9.39,US,1730644859
2,2,margaret river,-33.9500,115.0667,12.10,79,2,2.83,AU,1730644860
3,3,port elizabeth,-33.9180,25.5701,22.16,82,20,5.66,ZA,1730644862
4,4,punta arenas,-53.1500,-70.9167,11.06,58,75,7.72,CL,1730644580


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
hvplot.save(humidity_map, "output_data/humidity_map.html")
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df['Lat'] >= 13) & (city_data_df['Max Temp'] >= 20)]

# Drop any rows with null values
ideal_city_df = criteria_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
18,18,al ghayzah,16.2079,52.1760,28.18,68,1,4.44,YE,1730644881
22,22,inarajan village,13.2736,144.7484,26.54,93,100,3.42,GU,1730644885
30,30,princeton,25.5384,-80.4089,28.34,72,0,9.77,US,1730644894
31,31,tazacorte,28.6290,-17.9293,24.54,96,20,2.57,ES,1730644895
36,36,saipan,15.1355,145.7010,28.39,78,40,0.00,MP,1730644901
...,...,...,...,...,...,...,...,...,...,...
537,537,nasir,15.5894,32.5634,33.02,13,0,6.31,SD,1730645483
543,543,balimila,18.2500,82.1333,22.26,84,21,1.51,IN,1730645491
545,545,estelle,29.8458,-90.1067,26.67,72,40,4.63,US,1730645494
546,546,machico,32.7000,-16.7667,24.45,73,40,9.26,PT,1730645496


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name='')

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel_Name
18,al ghayzah,YE,16.2079,52.1760,68,
22,inarajan village,GU,13.2736,144.7484,93,
30,princeton,US,25.5384,-80.4089,72,
31,tazacorte,ES,28.6290,-17.9293,96,
36,saipan,MP,15.1355,145.7010,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
al ghayzah - nearest hotel: فندق تاج العرب
inarajan village - nearest hotel: No hotel found
princeton - nearest hotel: La Quinta Inn & Suites Miami Cutler Bay
tazacorte - nearest hotel: App Leyma
saipan - nearest hotel: Chalan Kanoa Beach Hotel
beas de segura - nearest hotel: Hotel Avenida
hawaiian paradise park - nearest hotel: No hotel found
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
pombas - nearest hotel: Aldeia Jerome
kingston - nearest hotel: Aphrodite Hotel
sayat - nearest hotel: No hotel found
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
kidal - nearest hotel: No hotel found
gatesville - nearest hotel: Main Street Inn
tela - nearest hotel: Hotel Plaza
veracruz - nearest hotel: No hotel found
ausa - nearest hotel: Can Pamplona
amguri - nearest hotel: No hotel found
balotra - nearest hotel: No hotel found
kapa'a - nearest hotel: Pono Kai Resort
peniche - nearest hotel: Residencial Maciel
babu - nearest hotel: 贺州国

,City,Country,Lat,Lng,Humidity,Hotel_Name,Hotel Name
18,al ghayzah,YE,16.2079,52.1760,68,,فندق تاج العرب
22,inarajan village,GU,13.2736,144.7484,93,,No hotel found
30,princeton,US,25.5384,-80.4089,72,,La Quinta Inn & Suites Miami Cutler Bay
31,tazacorte,ES,28.6290,-17.9293,96,,App Leyma
36,saipan,MP,15.1355,145.7010,78,,Chalan Kanoa Beach Hotel
...,...,...,...,...,...,...,...
537,nasir,SD,15.5894,32.5634,13,,منزل طارق
543,balimila,IN,18.2500,82.1333,84,,No hotel found
545,estelle,US,29.8458,-90.1067,72,,Baymont Marrero
546,machico,PT,32.7000,-16.7667,73,,Residence Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Lat",
    scale = 1,
    color = "City", 
    hover_cols = ["Hotel Name", "Country"]
)
# Display the map
hvplot.save(hotel_map, "output_data/hotel_map.html")
hotel_map 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)